# How to Select Values and Optimize Intel's MKL LINPACK Benchmark

## Problem Size

The Problem Size (N) should maximize the use of available memory while allowing for system operations and ensuring divisibility by the Block Size (NB). Given a total memory $M$ and a percentage of memory to be used $U$, the formula for $N$ can be approximated as follows, considering double precision (8 bytes per number):

$$
N = \text{floor}\left(\sqrt{\frac{M \times U \times 10^{9}}{8}}\right) - \left(\text{floor}\left(\sqrt{\frac{M \times U \times 10^{9}}{8}}\right) \mod NB\right)
$$

Where:
- $M$ = Total memory in Gigabytes
- $U$ = .7 to .8 (70 to 80% utilization)
- $NB$ = Block Size

This formula is designed to calculate the optimal size of the problem (N) that the High-Performance Linpack (HPL) benchmark can handle on a system with a specific amount of memory, taking into consideration both the total memory available and a specified utilization percentage. Let's break down the intuition behind each part of the formula:

The initial part of the formula,

$$
\sqrt{\frac{M \times U \times 10^{9}}{8}}
$$

calculates the size of the problem that can fit into the memory specified by $M$ (total memory in GiB) and $U$ (the fraction of total memory to be used). The factor $10^{9}$ converts GiB to bytes, and the divisor $8$ accounts for each double precision number consuming 8 bytes of memory. We take the square root because N is the dimension of one side of a matrix so we want a number that when squared gives us the total.

The floor function,

$$
\text{floor}(...)
$$

is used to ensure that the resulting problem size is an integer. Since we're dealing with discrete units (bytes, elements of the matrix), we can't have a fraction of an element.

The final adjustment,

$$
\text{floor}\left(\sqrt{\frac{M \times U \times 10^{9}}{8}}\right) \mod NB
$$

ensures that the calculated problem size is divisible by the block size (NB). We want our matrix to be chunked up into even blocks and if this number isn't divisible by block size you end up with a partial matrix that must be solved.

So, the subtraction of

$$
- \left(\text{floor}\left(\sqrt{\frac{M \times U \times 10^{9}}{8}}\right) \mod NB\right)
$$

from the floor of the square root of the memory calculation ensures that the final problem size (N) is both a realistic representation of what the system's memory can handle and is perfectly divisible by the chosen block size.

## Block Size (NB)

Block size is largely affected by computer architecture in combination with the matrix size and shape. There isn't a good way to precisely estimate this so you largely will check this value via experimentation. Good values typically range from 100-384. It is common on newer systems to start with:

$$
NB = 384
$$

## Process Grid (P x Q) On **Non-Intel MKL LINPACK**

This section is written if you are not using the Intel MKL LINPACK. If you are, these settings will not work.

The Process Grid dimensions should take into account the total number of cores and aim to distribute work evenly while considering physical architecture (like NUMA nodes). For a total of $C$ cores, the goal is to find $P$ and $Q$ such that $P \times Q = C$ and $|P - Q|$ is minimized. An initial approximation can be to set $P$ and $Q$ to the square root of $C$:

$$
P = Q = \sqrt{C}
$$

To be more exact, we have to make sure P and Q are whole numbers and generally P should be smaller than Q.

$$
P = 2 \times \lfloor \frac{\sqrt{C}}{2} \rfloor
$$

$$
Q = \left\{
    \begin{array}{ll}
        P & \text{if } P \times P \geq C \\
        P + 2 & \text{otherwise}
    \end{array}
\right.
$$

- **$ \sqrt{C} $**: The square root of the total number of cores ($C$) gives an initial estimate for the dimensions of a square process grid that would, in theory, equally distribute the workload across all cores. This is a common starting point for determining $P$ and $Q$ because it simplifies the assumption of an even distribution of tasks.
- **$2 \times \lfloor \frac{\sqrt{C}}{2} \rfloor$**: This calculation adjusts the initial estimate to ensure $P$ is an even number. We divide the initial estimate by $2$, round down to the nearest whole number (using the floor function, $\lfloor \cdot \rfloor$), and then multiply by $2$ again. This gets you the nearest whole number.
- **$P$ or $P + 2$**: The conditional logic for $Q$ addresses the total coverage of cores while also adhering to the constraint that $P \leq Q$ and both are even. The choice between $P$ and $P + 2$ ensures that:
  - $Q$ is at least as large as $P$, maintaining an even distribution of tasks.
  - If simply doubling $P$ does not cover all cores ($P \times P < C$), $Q$ is incremented by $2$ (to the next even number) to help cover the shortfall.
  - This adjustment maintains evenness and aims to utilize as many cores as possible without exceeding the total number of available cores. It also respects the preference for $P$ being smaller than $Q$ if they are not equal, which can be relevant for optimizing certain types of parallel computation that perform better with a "taller" rather than a "wider" grid.

**WARNING** This does not account for NUMA.

## N Estimator

I have written the below Python to help estimate the value for N specifically. It also gives P and Q, but these are less helpful since they don't apply as shown below to Intel's variant.

In [12]:
import math

def calculate_problem_size(total_memory_gb, NB, memory_utilization=0.5):
    bytes_per_double = 8
    memory_bytes = total_memory_gb * (10**9)
    
    # Calculate the maximum matrix size that can fit into the specified memory
    max_matrix_elements = memory_bytes * memory_utilization / bytes_per_double
    max_matrix_size = math.sqrt(max_matrix_elements)
    
    # Adjust N to be divisible by NB
    N = math.floor(max_matrix_size) - (math.floor(max_matrix_size) % NB)
    
    return N

def print_grid_block_and_problem_sizes(total_memory_gb, total_cores, NB_list):
    # Calculate problem sizes for each NB
    problem_sizes = [calculate_problem_size(total_memory_gb, NB) for NB in NB_list]
    
    # Assuming P and Q are calculated once as they are not dependent on NB
    sqrt_cores = math.sqrt(total_cores)
    P = 2 * math.floor(sqrt_cores / 2)  # Ensure P is even
    Q = P if P * P >= total_cores else P + 1  # Adjust Q if necessary
    
    # Print values of P, Q, NB, and N
    print("Values of P, Q, NB, and N:")
    print("P:", P)
    print("Q:", Q)
    print("NBs:", ', '.join(str(NB) for NB in NB_list))
    print("Ns:", ', '.join(str(N) for N in problem_sizes))

# Example usage
total_memory_gb = 1536  # Total memory in GB
total_cores = 336  # Total number of cores
NB_list = [192, 256, 384]  # List of NB values

print_grid_block_and_problem_sizes(total_memory_gb, total_cores, NB_list)

Values of P, Q, NB, and N:
P: 18
Q: 19
NBs: 192, 256, 384
Ns: 309696, 309760, 309504




## Intel MKL LINPACK Specific Values

If you're like me, you Google'd LINPACK and how to optimize. If you are also like me you landed on a bunch of open source documentation about how to tune it. Forget everything you read because that's not how the Intel version works.

Inside of [runme_intel64_dynamic](./binary/runme_intel64_dynamic) you will see the following values:

```bash
# Set total number of MPI processes for the HPL (should be equal to PxQ).
export MPI_PROC_NUM=2

# Set the MPI per node for each node.
# MPI_PER_NODE should be equal to 1 or number of sockets on the system.
# It will be same as -perhost or -ppn paramaters in mpirun/mpiexec.
export MPI_PER_NODE=2

# Set the number of NUMA nodes per MPI. (MPI_PER_NODE * NUMA_PER_MPI)
# should be equal to number of NUMA nodes on the system.
export NUMA_PER_MPI=1
```

These are the heart of Intel's benchmark. There is more detail on the entire process flow for intel in the [README](./README.md). Here is what each value does and how to tune it. For the sake of example, let's say your setup is 3 servers, each with 112 cores and 8 NUMA domains a piece.

- `MPI_PROC_NUM`: This is the total number of MPI processes that will run cluster wide. In my testing, I found that best performance was to have one MPI process per NUMA domain available. So if each server has 8, you would want 24 MPI processes total.
- `MPI_PER_NODE`: This controls how many MPI processes are allocated to each node in a round robin fashion. When I say round robin, I mean you could do something like set this in our example to 4 and what MPI would do is assign the first 4 MPI processes to node 1, 4 after to node 2, another 4 to node 3, and then it would round robin back to node 1 and add 4 again to each to get the full 24 processes. In my testing this did not yield optimal results. What is better is to simply set `MPI_PER_NODE` to the number of NUMA domains per server so in our case 8.
- `NUMA_PER_MPI`: This dictates how many NUMA domains each MPI process will span. If it is set to 1, each MPI process will only operate on a single NUMA domain, 2 each MPI process will span two NUMA domains. I show what this looks like in gory detail in the [runme_intel64_prv section of the README](./README.md#runme_intel64_prv). My best results were with this set to 1 and then I just had each MPI process bind to a single NUMA domain.

**CRITICAL** You must heed the comment "Set the number of NUMA nodes per MPI. (MPI_PER_NODE * NUMA_PER_MPI) should be equal to number of NUMA nodes on the system.". Whatever config you test, `MPI_PER_NODE*NUMA_PER_MPI=<number of NUMA nodes (on a single system)>`. While it would be suboptimal you **can** have `MPI_PER_NODE*NUMA_PER_MPI<number of system NUMA nodes`. That config will run. However, if `MPI_PER_NODE*NUMA_PER_MPI>number of system NUMA nodes`, the test will fail out with memory errors.

Finally, you will need to set `p` and `q`. Since Intel spawns threads automatically, `p` and `q` for Intel's variant is nothing like it is for regular MPI. This is one part with which you may want to experiment. The one key is that $p \times q = \text{MPI\_PROC\_NUM}$. I generally find best performance is when the numbers are as close as they can be. In our example, this with be $4 \times 6$ or maybe $2 \times 12$.

## Running the Intel Benchmark

1. Don't run under SLURM.
2. Seriously, I wouldn't run this under SLURM or any other job manager.

If you do, that's fine, but you will want to look at the script [mpirun](./binary/mpirun) to see how those variables are mapped to `-ppn` and `-np` for `mpiexec.hydra`. If you head this advice, then all you have to do to run the Intel LINPACK variant is:

1. Download the [LINPACK benchmark from Intel](https://www.intel.com/content/www/us/en/developer/articles/technical/onemkl-benchmarks-suite.html)
   1. **You will be using the  Intel® Distribution for MP LINPACK* Benchmark for Clusters** version! Do not confuse this with the "Intel® Distribution for LINPACK* Benchmark" version. (No I don't know who named them and yes, they should rethink that.) They aren't the same thing.
2. Distribute SSH keys so that you can login to everything without warnings
3. Go to this folder: `benchmarks_<BENCHMARK_VERSION>/linux/share/mkl/benchmarks/mp_linpack`
   1. Notice how it says `mp_linpack` at the end. `mp_linpack` is the one meant for multiple hosts.
4. Create a file called hosts with the hosts you want to run against laid out as shown below
5. Edit the `runme_intel64_dynamic` and go down to the line `mpirun -perhost ${MPI_PER_NODE} -np ${MPI_PROC_NUM} ./runme_intel64_prv "$@" | tee -a $OUT`. Something fun that isn't documented is that if you want to specify hosts here, which will then get passed to `mpirun`... which will then get passed to `mpiexec.hydra`... which then actually uses the hosts file. How would you know that without reverse engineering the code base I hear you asking? You wouldn't! That's the fun part! Not that I resent how much time it took me to figure out how all this is glued together. I definitely don't. Promise. Anyway... so you need to change that to `mpirun -hostfile hosts -perhost ${MPI_PER_NODE} -np ${MPI_PROC_NUM} ./runme_intel64_prv "$@" | tee -a $OUT`
6. Now you're ready to run. Execute `./runme_intel64_dynamic  -n <YOUR_N> -b <YOUR_BLOCK_SIZE> -p 4 -q 6`. If you are using a machine file, you need to edit the `runme_intel64_dynamic` file and add the `-machinefile` argument.

**hosts**
```
<ssh_username>@host1
<ssh_username>@host2
<ssh_username>@host3
```


## Optimizing

I wrote the script [get_node_utilization](./get_node_utilization.py) to help with optimizing. Here is an example of the output for a run on node 1 using our example of three servers, 8 NUMA domains each, 112 cores. This grabs all of the `xhpl_intel64_dynamic` processes and threads and lists them along with their CPU affinity and assigned NUMA nodes.

You may notice the processes that have low utilization. Ex: `191817     0.01       0-13            0`. Those processes are the parent processes that spawn the worker threads.

```bash
PID: 191802, Name: xhpl_intel64_dynamic, Threads:
Thread ID  CPU Time   CPU Affinity    NUMA Node(s)   
-----------------------------------------------------
191802     499.66     0               0              
191817     0.01       0-13            0              
191819     494.13     13              0              
191821     494.15     2               0              
191835     494.15     3               0              
191843     494.18     4               0              
191851     494.18     5               0              
191853     493.99     6               0              
191854     493.42     7               0              
191856     494.13     8               0              
191859     494.10     9               0              
191862     494.12     10              0              
191868     493.93     11              0              
191875     494.10     12              0              
191918     499.15     1               0              

PID: 191803, Name: xhpl_intel64_dynamic, Threads:
Thread ID  CPU Time   CPU Affinity    NUMA Node(s)   
-----------------------------------------------------
191803     499.80     28              2              
191815     0.05       28-41           2              
191825     494.08     41              2              
191831     494.03     30              2              
191839     494.11     31              2              
191847     494.10     32              2              
191866     494.08     33              2              
191874     494.12     34              2              
191882     494.11     35              2              
191888     494.13     36              2              
191894     494.08     37              2              
191900     494.12     38              2              
191905     494.08     39              2              
191911     494.13     40              2              
191917     499.30     29              2              

PID: 191804, Name: xhpl_intel64_dynamic, Threads:
Thread ID  CPU Time   CPU Affinity    NUMA Node(s)   
-----------------------------------------------------
191804     500.00     98              7              
191812     0.03       98-111          7              
191827     489.41     111             7              
191833     490.17     100             7              
191841     490.44     101             7              
191849     489.57     102             7              
191869     489.55     103             7              
191878     489.84     104             7              
191884     490.37     105             7              
191890     489.51     106             7              
191896     490.28     107             7              
191902     490.16     108             7              
191908     489.85     109             7              
191912     490.55     110             7              
191919     499.48     99              7              

PID: 191805, Name: xhpl_intel64_dynamic, Threads:
Thread ID  CPU Time   CPU Affinity    NUMA Node(s)   
-----------------------------------------------------
191805     499.72     56              4              
191813     0.02       56-69           4              
191824     491.77     69              4              
191830     491.31     58              4              
191838     491.70     59              4              
191845     491.90     60              4              
191863     491.40     61              4              
191871     491.37     62              4              
191879     491.82     63              4              
191886     491.38     64              4              
191892     491.54     65              4              
191897     491.41     66              4              
191904     491.50     67              4              
191910     491.82     68              4              
191920     499.33     57              4              

PID: 191806, Name: xhpl_intel64_dynamic, Threads:
Thread ID  CPU Time   CPU Affinity    NUMA Node(s)   
-----------------------------------------------------
191806     499.76     42              3              
191810     0.04       42-55           3              
191822     494.05     55              3              
191828     493.99     44              3              
191836     494.09     45              3              
191844     494.07     46              3              
191858     493.97     47              3              
191865     494.05     48              3              
191873     494.04     49              3              
191881     494.04     50              3              
191887     494.08     51              3              
191893     494.04     52              3              
191899     494.05     53              3              
191906     494.07     54              3              
191916     499.20     43              3              

PID: 191807, Name: xhpl_intel64_dynamic, Threads:
Thread ID  CPU Time   CPU Affinity    NUMA Node(s)   
-----------------------------------------------------
191807     500.05     84              6              
191814     0.05       84-97           6              
191826     489.49     97              6              
191832     489.61     86              6              
191840     489.08     87              6              
191848     490.11     92              6              
191867     489.12     89              6              
191876     490.13     90              6              
191885     489.29     91              6              
191891     489.67     88              6              
191898     489.57     93              6              
191903     489.40     94              6              
191909     489.73     95              6              
191913     490.22     96              6              
191914     499.51     85              6   
PID: 191808, Name: xhpl_intel64_dynamic, Threads:
Thread ID  CPU Time   CPU Affinity    NUMA Node(s)   
-----------------------------------------------------
191808     499.96     70              5              
191816     0.03       70-83           5              
191823     491.61     83              5              
191829     492.17     72              5              
191837     491.24     73              5              
191846     491.89     74              5              
191861     492.41     75              5              
191870     491.58     76              5              
191877     491.99     77              5              
191883     492.03     78              5              
191889     492.09     79              5              
191895     491.91     80              5              
191901     491.73     81              5              
191907     491.63     82              5              
191921     499.39     71              5              

PID: 191809, Name: xhpl_intel64_dynamic, Threads:
Thread ID  CPU Time   CPU Affinity    NUMA Node(s)   
-----------------------------------------------------
191809     499.77     14              1              
191811     0.05       14-27           1              
191818     494.07     27              1              
191820     494.06     16              1              
191834     494.00     17              1              
191842     494.10     18              1              
191850     494.06     19              1              
191852     494.09     20              1              
191855     494.10     21              1              
191857     494.10     22              1              
191860     494.11     23              1              
191864     494.08     24              1              
191872     494.08     25              1              
191880     494.12     26              1              
191915     499.30     15              1  
```